In [40]:
import redis
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import time
import datetime
import pandas as pd

# Setup
functions for connecting and setting up the databases 

In [41]:
current_env = 'production' # 'docker' for development on docker, 'production' for real server
current_user = 'stud10'
ENV = {"docker": '172.17.0.1',"production":'bdl1.eng.tau.ac.il'}
companiesSet = current_user+":company:names"
open_jobs = current_user+":oj:"
cand_app_set = current_user+":candidate_application"
redis_db_num = 10

### Connections

In [42]:
def connect_to_redis(db_num=redis_db_num,port=6379):
    # We connect to redis on a specific database (db_num) to not interuprt other students
    r = redis.StrictRedis(host=ENV[current_env], port=port, socket_connect_timeout=10,db=db_num)
    r.ping() # send ping to verify that a connection established to redis
    print('connected to redis on "{}", port {}'.format(ENV[current_env], port)) 
    return r

def connect_to_mongo(port=27017):
    if current_env == 'docker':
        client = MongoClient(host=ENV[current_env],port=port, connectTimeoutMS=10000)
        # client.admin.command('ping') # try to ping mongo server
        print('connected to MongoDB on "{}", port {}'.format(ENV[current_env], port))
    
    elif current_env == 'production':
        client = MongoClient()
    return client


### Initializiation 

In [43]:
def setup_mongo(client):
    db = client[current_user]
    companies = db.companies
    print("companies collection created in {} db".format(current_user))
    return db, companies

def reset_redis(r):
    r.flushdb()
    print('redis database number {} is clean'.format(redis_db_num))

def reset_mongo(client):
    client[current_user].companies.drop()
    print("mongo is clean")

def restart():
    # connect to dbs
    r = connect_to_redis()
    client = connect_to_mongo()
    # clean dbs
    reset_redis(r)
    reset_mongo(client)
    # setup the databases
    db, companies = setup_mongo(client)
    print("restart completed")
    return r, client, db, companies

# Operation 1 - Add a new company

In [44]:
def is_company_exists(r, companies, company_name):
    # check if company name in comapny set on redis
    if r.sismember(companiesSet, company_name):
        return True
    
    # if not, verify against mongo
    elif companies.find_one({"company_name": company_name}) is not None:
        return True
    
    else:
        return False
    
def add_company(company_dict,r=None, companies=None):
    if r is None:
        r = connect_to_redis()
    if companies is None:
        companies = connect_to_mongo()[current_user].companies
        
    # getting dict for the company and insert it to db
    if 'company_name' not in company_dict:
        # raise ValueError("company dict must have company_name field") #no errors allowed
        print("company dict must have company_name field, no changes were commited to db")
        return None

    else:
        company_name = company_dict['company_name']
    
    # verify that company name is unique
    if(is_company_exists(r, companies, company_name)):
        # raise ValueError("company name already taken") #no errors allowed
        print("company name already taken, no changes were commited to db")
        return None
        
    # and then insert to mongo and redis
    company_dict['jobs_list'] = [] # initiate jobs list
    companies.insert_one(company_dict)
    r.sadd(companiesSet, company_dict['company_name'])
    print("%s Added successfully!"%(company_name))

# Operation 2 - Add a new job position

In [56]:
def generate_job_id(companies, company_name):
    match = {'$match' : {'company_name':company_name}}
    project = {'$project': { 'max_id': { '$size':'$jobs_list' }}}
    res = companies.aggregate([match,project])
    return list(res)[0]['max_id'] + 1

def add_job(job_dict, company_name, r=None, companies=None):
    # ASSUMPTION: there is no option to delete jobs (so counting jobs can be used to generate job id)
    if r is None:
        r = connect_to_redis()
    if companies is None:
        companies = connect_to_mongo()[current_user].companies
    if not (is_company_exists(r, companies, company_name)):
        print("company doesn't exist")
        return
    
    # generate job id and insert to the company object
    job_dict['job_id'] = generate_job_id(companies, company_name)
    job_dict['application_list'] = [] # initiate application list
    companies.update_one({'company_name': company_name}, {'$push': {'jobs_list': job_dict}}, upsert = True)
    # add job to redis
    if job_dict['status'] == 'open':
        title = job_dict['job_title']
        location = job_dict['location']
        key_string = open_jobs + "%s:%s"%(location,title)
        res = r.incr(key_string)
        print('There are now {} open jobs for: {} in {}'.format(res,title,location))
    
    print("job with id %s was added to %s jobs successfully!"%(job_dict['job_id'], company_name))

# Operation 3 - Add a new application

In [46]:
def is_job_open(companies, company_name, job_id):
    res = companies.find_one(
        { 
            "company_name": company_name,
        },
        { "jobs_list": { "$elemMatch": { "job_id": int(job_id), "status": "open" }}}
    )
    return('jobs_list' in res)

def is_already_submitted(companies, company_name, job_id, email):
    res = companies.find_one(
        { 
            "company_name": company_name,
        },
        { "jobs_list": { "$elemMatch": { "job_id": int(job_id), "application_list": {"$elemMatch": {'email':email}} }}}
    )
    return('jobs_list' in res) # if mail doesn't exists an empty object will returned

def new_application(candidate, application_time, job_id, company_name, r=None, companies=None):
    if r is None:
        r = connect_to_redis()
    if companies is None:
        companies = connect_to_mongo()[current_user].companies

    if (not is_job_open(companies, company_name, job_id)):
        print("you are trying to apply to a closed job")
        return -1

    if (is_already_submitted(companies, company_name, job_id, candidate['email'])):
        print("you have already sent application for this job")
        return -2

    # update in mongo
    d = datetime.datetime.strptime(application_time, "%d-%m-%Y %H:%M:%S")
    candidate['application_date'] = d
    companies.update_one({"company_name": company_name,"jobs_list": {"$elemMatch":{"job_id":int(job_id)}}},{'$push':{'jobs_list.$.application_list':candidate}})

    #update in redis
    unix_d = time.mktime(d.timetuple())
    #check if company already exists in redis
    key_string = "candidate_applications:{}".format(candidate['email'])
    comps = r.zrange(key_string,0,-1,withscores=True)
    comp = [x for x in comps if x[0]==company_name]
    # if comany already exists - change vlaue to most recent date
    if len(comp)>0:
        old_timestamp = comp[0][1]
        if unix_d>old_timestamp:
            r.zadd(key_string,unix_d,company_name)
    else:
        r.zadd(key_string,unix_d,company_name)
        
    print("{} submited application for job number {} at {}".format(candidate['candidate_name'],job_id, company_name))

# Operation 4 - Update job status

In [61]:
def update_job_status(company_name, job_id, new_status, r=None, companies=None):
    res = companies.aggregate([
    { "$unwind": "$jobs_list"},
    { "$match": {"jobs_list.job_id": 1, 'company_name':'TAU'}},
    { "$project": { "jobs_list.status" : 1, 'jobs_list.location': 1, 'jobs_list.job_title': 1}}
    ])

    for job in res:
        old_status = job['jobs_list']['status']
        location = job['jobs_list']['location']
        job_title = job['jobs_list']['job_title']
    
    #update redis
    key_string = open_jobs + "%s:%s"%(location,job_title)
    if old_status == 'open' and new_status == 'close':
        res = r.decr(key_string)
    
    elif new_status == 'open' and old_status == 'close':
        res = r.incr(key_string)
    
    #update mongo
    companies.update_one(
        {"company_name": company_name,
        "jobs_list": {
            "$elemMatch":{"job_id":int(job_id)}}}, 
        {'$set':{'jobs_list.$.status':new_status}}
        )
    
    print("job number: {} at {} is now: {}".format(job_id, company_name, new_status))
    print('There are now {} open jobs for: {} in {}'.format(res, job_title,location))

# Operation 5 - show latest companies

In [48]:
def show_latest_10_companies(candidate_email):
    key_string = "candidate_applications:{}".format(candidate_email)
    return r.zrevrange(key_string, 0, 9, withscores=False)

# Operation 6 - show number of open jobs

In [49]:
def show_number_of_jobs(location,title):
    key_string = open_jobs + "%s:%s"%(location,title)
    result = r.get(key_string)

# Report 1
This functions returns ... <br>
We tried sorting values in mongo instead of in pandas, but it didn't work properly. We believe it is due to old version of. mongo. We left commented code in solution so you can see we tried.

In [50]:
def count_jobs_by_company(companies):
    project = {
    '$project':{
        'company_name':1,
        '_id':0,
        'open_jobs':{
            '$size':{
                '$filter' :{
                    'input': '$jobs_list',
                    'as':'oj',
                    'cond':{'$eq':['$$oj.status','open']}
                    }
                }
            },
        'closed_jobs':{
            '$size':{
                '$filter' :{
                    'input': '$jobs_list',
                    'as':'cj',
                    'cond':{'$eq':['$$cj.status','close']}
                    }
                }
            }
        }
    }
    # sort1 = {'$sort':{'open_jobs':-1,'closed_jobs':-1}} #this doesn't work as expected
    # res = companies.aggregate([project,sort])
    res = companies.aggregate([project])
    df = pd.DataFrame(list(res))
    
    try:
        # arange column names for aesthitcal reasons 
        ordered_cols = ['company_name','open_jobs','closed_jobs']
        df = df[ordered_cols]
        # sort values
        df.sort_values(by=['open_jobs','closed_jobs'],ascending=False,inplace=True)
    except KeyError:
        print('No job at any company exists - no records returned') 
    # return list(res)
    return df

In [51]:
# count_jobs_by_company(companies)

# Report 2

In [52]:
def count_candidates_by_job():
    unwind = {'unwind'}
    project = {
        '$project':{
            '_id':0,
        }
    }  
    # sort1 = {'$sort':{'open_jobs':-1,'closed_jobs':-1}} #this doesn't work as expected
    res = companies.aggregate([project,sort])
    # res = companies.aggregate([project])
    df = pd.DataFrame(list(res))
    
    try:
        # arange column names for aesthitcal reasons 
        ordered_cols = ['company_name','open_jobs','closed_jobs']
        df = df[ordered_cols]
        # sort values
        # df.sort_values(by=['open_jobs','closed_jobs'],ascending=False,inplace=True)
    except KeyError:
        print('No job at any company exists - no records returned') 
    # return list(res)
    return df

# Run all operations

In [62]:
r = connect_to_redis(redis_db_num)
client = connect_to_mongo()

# clean the databases
reset_redis(r)
reset_mongo(client)

# setup the databases
db, companies = setup_mongo(client)

connected to redis on "bdl1.eng.tau.ac.il", port 6379
redis database number 10 is clean
mongo is clean
companies collection created in stud10 db


In [63]:
# Operation 1 - Add a new company
add_company({'company_name':'TAU', 'company_description':'University'}, r=r, companies=companies)

TAU Added successfully!


In [64]:
# Operation 2 - Add a new job position
add_job({'job_title':'bi developer', 'location': 'Tel Aviv','requirements':['python','big data','mongodb'],'status':'open','publish_date':'01-02-2020'},'TAU', r=r, companies=companies)

there are now 1 open jobs for: bi developer in Tel Aviv
job with id 1 was added to TAU jobs successfully!


In [65]:
# Operation 3 - Add a new application
new_application({'candidate_name':'laura', 'email':'laura@gmail.com','linkedin':'https://www.linkedin.com/in/laura/', 'skills': ['python','sql']},'01-02-2020 15:00:00', '1','TAU', r=r, companies=companies)

laura submited application for job number 1 at TAU


In [66]:
# Operation 4 - Update job status
update_job_status('TAU','1','close', r=r, companies=companies)

job number: 1 at TAU is now: close
there are now 0 open jobs for: bi developer in Tel Aviv


# Tests 

In [33]:
count_jobs_by_company(companies)

,company_name,open_jobs,closed_jobs
1,motiluhim_inc,2,2
0,TAU,1,3


In [32]:
add_company({'company_name':'motiluhim_inc', 'company_description':'itzikspensiv'}, r=r, companies=companies)
add_job({'job_title':'fullstack developer', 'location': 'Tel Aviv','requirements':['python','big data','mongodb'],'status':'close','publish_date':'01-02-2020'},'motiluhim_inc', r=r, companies=companies)
add_job({'job_title':'kaki developer', 'location': 'Tel Aviv','requirements':['python','big data','mongodb'],'status':'open','publish_date':'01-02-2020'},'motiluhim_inc', r=r, companies=companies)

company name already taken, no changes were commited to db
job with id 3 was added to motiluhim_inc jobs successfully!
job with id 4 was added to motiluhim_inc jobs successfully!


In [30]:
add_job({'job_title':'frontend developer', 'location': 'Tel Aviv','requirements':['python','big data','mongodb'],'status':'close','publish_date':'01-02-2020'},'TAU', r=r, companies=companies)

job with id 4 was added to TAU jobs successfully!


In [23]:
update_job_status('motiluhim_inc','1','open', r=r, companies=companies)

job number: 1 at motiluhim_inc is now: open


In [48]:

project = {
'$project':{
    'company_name':1,
    '_id':0,
    'open_jobs':{
        '$size':{
            '$filter' :{
                'input': '$jobs_list',
                'as':'oj',
                'cond':{'$eq':['$$oj.status','open']}
                }
            }
        },
    'closed_jobs':{
        '$size':{
            '$filter' :{
                'input': '$jobs_list',
                'as':'cj',
                'cond':{'$eq':['$$cj.status','close']}
                }
            }
        }
    }
}
sort = {'$sort':{'open_jobs':-1,'closed_jobs':-1}} #this doesn't work as expected
res = companies.aggregate([project,sort])
# res = companies.aggregate([project])
df = pd.DataFrame(list(res))


df

,closed_jobs,company_name,open_jobs
0,3,TAU,1
1,2,motiluhim_inc,2


In [86]:
unwind = {
    '$unwind':'$jobs_list'
}
group = {
    '$group':{'_id':'$comapny_name'}
}
sbc = {
    '$sortByCount':'$jobs_list.status'
}

In [87]:
zibi = companies.aggregate([unwind,group])
pd.DataFrame(list(zibi))

,_id
0,None


In [89]:
companies['jobs_list'].aggregate([
    {'$group':{
        '_id':'$status',
        'count':{'$sum':1}
    }}
])

[]

In [ ]:
pd.DataFrame(list(res))
# list(res)

,closed_jobs,company_name,open_jobs
0,1,TAU,1
1,0,motiluhim_inc,2


In [ ]:
res=companies.find_one({"company_name": 'motiluhim_inc'})
list(res['jobs_list'])

[{u'status': u'open', u'requirements': [u'python', u'big data', u'mongodb'], u'job_id': 1, u'application_list': [], u'publish_date': u'01-02-2020', u'location': u'Tel Aviv', u'job_title': u'fullstack developer'}, {u'status': u'open', u'requirements': [u'python', u'big data', u'mongodb'], u'job_id': 2, u'application_list': [], u'publish_date': u'01-02-2020', u'location': u'Tel Aviv', u'job_title': u'kaki developer'}]

In [ ]:
new_application({'candidate_name':'laura', 'email':'laura@gmail.com','linkedin':'https://www.linkedin.com/in/laura/', 'skills': ['python','sql']},'14-02-2022 15:00:00', '3','TAU', r=r, companies=companies)

you are trying to apply to a closed job


-1

In [ ]:
r.zrange('candidate_applications:laura@gmail.com',0,-1,withscores=True)

[('TAU', 1580562000.0)]

In [ ]:
a = r.zrange('moti:luhim',0,-1,withscores=True)
b = [x for x in a if x[0]=='haim']
b

[]

In [22]:
a=1 
b=2 
c=3 
print('{}+{}+{}'.format(a, b, c))

1+2+3
